# Answering Business Questions In SQL

We will be using a fictional database called chinook. It describes the databases within an organisation selling music through the iTunes store. 
The database includes employee details, who reports to which manager, customer information and a wealth of information of the tracks purchased (such as playlists, album, artist, genre and media type)

## Aims

To analyse the chinnook employee data base and provide important information that aids business decisions

## Introduction
First lets view the schema. This is a graphical display of the tables available to us. 

The table names are in bold. The lines indicate matching columns and show the table relationships

In [1]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://s3.amazonaws.com/dq-content/191/chinook-schema.svg")

Setting up SQL

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [3]:
%%sql /*#An example of the tables*/
SELECT *
FROM employee
LIMIT 3;

Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com


## Chinooks World Market

Lets have a look at the market as a whole to get a feel of how Chinook is performing in the various country markets.

First a check of the data to be sure we dont have billing in one country and the customer in another.

In [4]:
%%sql
SELECT *
FROM invoice AS i
LEFT JOIN customer AS c
ON c.customer_id = i.customer_id
WHERE c.country IS NOT i.billing_country

Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total,customer_id_1,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id


Purchases across countries has not occured in this dataset. 

Next we shall look at how many tracks are in the world per country and what percentage this is of the global market.
Also knowing that tracks have wide range of values, we will workout the total value of these markets.

In [5]:
%%sql
SELECT  billing_country AS Country,
        SUM(il.quantity) AS Number_of_tracks_sold,
        ROUND((SUM(CAST(il.quantity AS FLOAT)/(SELECT CAST(COUNT(*) AS FLOAT)
            FROM invoice_line)) *100),2)
            AS Percentage_of_tracks
        
FROM invoice AS i
LEFT JOIN invoice_line as il
ON il.invoice_id = i.invoice_id
GROUP BY 1
ORDER BY Percentage_of_tracks DESC

Done.


Country,Number_of_tracks_sold,Percentage_of_tracks
USA,1051,22.09
Canada,541,11.37
Brazil,432,9.08
France,393,8.26
Germany,338,7.11
Czech Republic,276,5.8
United Kingdom,248,5.21
Portugal,187,3.93
India,185,3.89
Ireland,116,2.44


USA, Canada, Brazil and France make up half of the music market.
Most tracks purchased were in the USA.

In general the less tracks the lower the market share. 
But this is not always correct. Lets look at the average track price per country.

In [6]:
%%sql
SELECT  billing_country AS Country,
        ROUND((SUM(total)/SUM(il.quantity)), 2) AS Average_track_price
FROM invoice AS i
LEFT JOIN invoice_line as il
ON il.invoice_id = i.invoice_id
GROUP BY billing_country
ORDER BY Average_track_price DESC

Done.


Country,Average_track_price
Ireland,12.36
Czech Republic,11.54
Australia,11.47
Spain,10.87
Hungary,10.51
India,10.21
Germany,10.18
Canada,10.15
France,10.11
United Kingdom,10.08


Customers from Ireland paid most per track for their music.
Customers from Denmark paid the least.
There are many factors affecting prices of the tracks like tax, the wealth of the customers, and if the music they bought just happened to be differently priced as it is different music.
There is no discernable trend here though.

## Chinook seeks USA contract advice

We are told that Chinook has signed a deal with a new record label.
The task is to select 3 unreleased albums to promote to the USA market.
They gave the following potential candidates.

| Artist Name | Genre |
| --- | --- | --- |
| Regal | Hip-Hop |
| Red Tone | Punk |
| Meteor and the Girls | Pop |
| Slim Jim Bites | Blues |

First lets create a new table that brings together information we might need.
The invoice totals and the genres of tracks bought in the USA

In [7]:
%%sql
CREATE VIEW USA_invoice_and_line_and_track_and_genre AS
SELECT *,
        genre.name AS Genre
FROM invoice
LEFT JOIN invoice_line ON invoice.invoice_id = invoice_line.invoice_id
LEFT JOIN track ON track.track_id = invoice_line.track_id
LEFT JOIN genre ON genre.genre_id = track.genre_id
WHERE invoice.billing_country = 'USA'

(sqlite3.OperationalError) table USA_invoice_and_line_and_track_and_genre already exists
[SQL: CREATE VIEW USA_invoice_and_line_and_track_and_genre AS
SELECT *,
        genre.name AS Genre
FROM invoice
LEFT JOIN invoice_line ON invoice.invoice_id = invoice_line.invoice_id
LEFT JOIN track ON track.track_id = invoice_line.track_id
LEFT JOIN genre ON genre.genre_id = track.genre_id
WHERE invoice.billing_country = 'USA']
(Background on this error at: http://sqlalche.me/e/13/e3q8)


Now lets collect the Genre and Market Value of these USA tracks and find our top value genres.

In [8]:
%%sql
SELECT  Genre,
        ROUND(SUM(total),2) AS Market_value
FROM USA_invoice_and_line_and_track_and_genre
GROUP BY 1
ORDER BY 2 DESC;

Done.


Genre,Market_value
Rock,5568.75
Alternative & Punk,1234.53
Metal,1025.64
R&B/Soul,633.6
Blues,453.42
Alternative,369.27
Hip Hop/Rap,366.3
Pop,201.96
Latin,162.36
Jazz,125.73


From highest to lowest market value in the USA:
Punk, Blues, Hip-Hop, Pop

As we are to pick 3 artists we will pick those with the genres of Punk, Blues and Hip-Hop

## Chinook employee assessments
The task is to assess the value of the employees.
The obvious link its to list the total value of the purchases made with the employee.
But first lets look at the employee table to see if there are any other factors

In [9]:
%%sql
SELECT *
FROM employee;

Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
6,Mitchell,Michael,IT Manager,1,1973-07-01 00:00:00,2016-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
7,King,Robert,IT Staff,6,1970-05-29 00:00:00,2017-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
8,Callahan,Laura,IT Staff,6,1968-01-09 00:00:00,2017-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


There are several factors that can affect the sales an employee has made.
- title - Only certain roles in the company are going to be selling, some have more admin or managerial positions which are important and cannot be justified based soley on revenue
- country - As we have seen earlier the average price per track varies per country. Do certain employees have different sectors? 
Is this total value from these a result of the employee, the countries economics or customer preference?
- hire_date - Clearly a new employee will have had less opportunity to generate revenue. Perhaps there is a period training were no sales can be made.

First lets review how many job titles there are in Chinook and how many customers were assinged to each

### Employee Assessment By Title

In [10]:
%%sql
SELECT e.title,
        count(c.customer_id) AS number_of_customers
FROM employee AS e
LEFT JOIN customer AS c ON e.employee_id = c.support_rep_id
GROUP BY e.title;

Done.


title,number_of_customers
General Manager,0
IT Manager,0
IT Staff,0
Sales Manager,0
Sales Support Agent,59


Perhaps this was in someways expected. No one in IT or a manager is making any sales.
Only the 3 sales support agents are.

For these 3 support agents lets see where there customers are based.

In [11]:
%%sql
CREATE VIEW reps_and_countries
AS SELECT  c.country,
        e.sales_rep_name
FROM customer AS c
LEFT JOIN (SELECT first_name || ' ' || last_name AS sales_rep_name,
                   employee_id
            FROM employee
            WHERE title ='Sales Support Agent') AS e
    ON e.employee_id = c.support_rep_id
ORDER BY 1

(sqlite3.OperationalError) table reps_and_countries already exists
[SQL: CREATE VIEW reps_and_countries
AS SELECT  c.country,
        e.sales_rep_name
FROM customer AS c
LEFT JOIN (SELECT first_name || ' ' || last_name AS sales_rep_name,
                   employee_id
            FROM employee
            WHERE title ='Sales Support Agent') AS e
    ON e.employee_id = c.support_rep_id
ORDER BY 1]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


First we shall see how many countries each sales rep has customers for.

### Employee Assessment By Countries of their Customers

In [12]:
%%sql
SELECT sales_rep_name,
        COUNT (DISTINCT country)
FROM reps_and_countries
GROUP BY sales_rep_name;

Done.


sales_rep_name,COUNT (DISTINCT country)
Jane Peacock,10
Margaret Park,12
Steve Johnson,13


Number of countries in not the whole story as some countries have higher revenues.
Lets see who has customers belonging to the countries in the top 10 markets

In [13]:
%%sql
SELECT *
FROM reps_and_countries
WHERE COUNTRY IN ('USA','Canada','Brazil','France','Germany','Czech Republic','United Kingdow', 'Portugal','India','Ireland')
GROUP BY country,sales_rep_name;

Done.


country,sales_rep_name
Brazil,Jane Peacock
Brazil,Margaret Park
Brazil,Steve Johnson
Canada,Jane Peacock
Canada,Margaret Park
Canada,Steve Johnson
Czech Republic,Margaret Park
Czech Republic,Steve Johnson
France,Jane Peacock
France,Margaret Park


All sales reps are selling to the top 4 markets (USA, Canada, Brazil and France)
For the top 10 countries:

Jane Peacock has 7

Margaret Park has 6

Steve Johnson has 6

The split of the countries assigned to sales representatives appears to be quite even.

### Employee Assessment by Length of Service

In [14]:
%%sql
SELECT first_name || ' ' || last_name AS sales_rep_name,
        hire_date
FROM employee
WHERE sales_rep_name IN ('Jane Peacock','Margaret Park', 'Steve Johnson') 
ORDER BY 2;

Done.


sales_rep_name,hire_date
Jane Peacock,2017-04-01 00:00:00
Margaret Park,2017-05-03 00:00:00
Steve Johnson,2017-10-17 00:00:00


All started in the same year but Jane has the longest length of service. 

It would be interesting to see the salary compared to revenue total, but for now we shall just compare total revenue to the employee.

In [15]:
%%sql
SELECT  e.sales_rep_name,
        ROUND(SUM(i.total),2)
        FROM customer AS c
LEFT JOIN (SELECT first_name || ' ' || last_name AS sales_rep_name,
                   employee_id           
            FROM employee
            WHERE title ='Sales Support Agent') AS e
    ON e.employee_id = c.support_rep_id
LEFT JOIN (SELECT total,
                  customer_id
           FROM invoice) AS i
    ON i.customer_id = c.customer_id
GROUP BY 1

Done.


sales_rep_name,"ROUND(SUM(i.total),2)"
Jane Peacock,1731.51
Margaret Park,1584.0
Steve Johnson,1393.92


The length of service appears to be slightly correlated with revenue produced and there doesnt seem to be evidence of underperforming from sales alone.

## Analysing the market

### Single customer markets
Just how big are the markets with single customers?

First lets see how many markets have a single customer

In [16]:
%%sql
DROP VIEW Markets_multi_and_single

Done.


[]

In [17]:
%%sql
CREATE VIEW Markets_multi_and_single AS
SELECT  c.country AS Country,
        ROUND(SUM(total), 2) AS Countries_market_value,
        COUNT (DISTINCT c.customer_id) AS Number_of_Customers
FROM customer AS c
LEFT JOIN invoice as i
ON i.customer_id = c.customer_id
GROUP BY 1
ORDER BY 2 DESC;

Done.


[]

In [18]:
%%sql
SELECT *
FROM Markets_multi_and_single

Done.


Country,Countries_market_value,Number_of_Customers
USA,1040.49,13
Canada,535.59,8
Brazil,427.68,5
France,389.07,5
Germany,334.62,4
Czech Republic,273.24,2
United Kingdom,245.52,3
Portugal,185.13,2
India,183.15,2
Ireland,114.84,1


Lets now make name those single customer markets as Other in a Region column.

In [19]:
%%sql
SELECT Region,
       ROUND(SUM(Countries_market_value),2),
       ROUND((SUM(CAST(Countries_market_value AS FLOAT)))
             / SUM(CAST(Number_of_Customers AS FLOAT))
             ,2)
        AS Sale_per_customer
FROM (SELECT *,
          CASE
             WHEN Number_of_Customers = 1 THEN 1
             ELSE 0
          END AS Sort,
          CASE
              WHEN Number_of_Customers = 1 THEN "Other"
              ELSE Country
          END AS Region
    FROM Markets_multi_and_single)
GROUP BY 1
ORDER BY sort;

Done.


Region,"ROUND(SUM(Countries_market_value),2)",Sale_per_customer
Brazil,427.68,85.54
Canada,535.59,66.95
Czech Republic,273.24,136.62
France,389.07,77.81
Germany,334.62,83.66
India,183.15,91.58
Portugal,185.13,92.56
USA,1040.49,80.04
United Kingdom,245.52,81.84
Other,1094.94,73.0


The 'Other's market is significant with a comparable market value to the biggest market - The USA

### What proportion of the market are Individual tracks and whole albums?

Chinook would like to explore the option of licensing the most popular individual tracks from an Artist as opposed to the whole albumn.
So which is the bigger market?

In [20]:
%%sql 
WITH track_invoice AS
(SELECT t.album_id, il.track_id, il.invoice_id
FROM track AS t
INNER JOIN invoice_line AS il ON il.track_id = t.track_id)

SELECT purchase_type,
        COUNT(DISTINCT invoice_id) AS Number_Of_Invoices,
        ROUND((CAST (COUNT(DISTINCT invoice_id) AS FLOAT)*100)/ 
            (SELECT count(distinct invoice_id) FROM invoice),2) AS Percentage

FROM
(SELECT invoice_id,
        CASE
        WHEN (SELECT t2.track_id /*#These are tracks for the whole album*/
             FROM track AS t2
             WHERE t2.album_id = track_invoice2.album_id
             EXCEPT
             SELECT il2.track_id  /*#These are only the purchased tracks*/
             FROM invoice_line AS il2
             WHERE il2.invoice_id = track_invoice2.invoice_id
             ) IS NULL
            AND 
/*# Combining the reverse of these CASE statements will label each invoice*/
/*#where the purchased tracks in the invoice was the whole album*/
            (SELECT il2.track_id  
             FROM invoice_line AS il2
             WHERE il2.invoice_id = track_invoice2.invoice_id
             EXCEPT SELECT t2.track_id 
             FROM track AS t2
             WHERE t2.album_id = track_invoice2.album_id
            ) IS NULL
            THEN 'Album'
            ELSE 'Individual Tracks'
            END AS purchase_type
    FROM track_invoice As track_invoice2
    )
GROUP BY purchase_type

Done.


purchase_type,Number_Of_Invoices,Percentage
Album,114,18.57
Individual Tracks,500,81.43


This is repeated in the next code just without using the except function.

In [21]:
%%sql
/*#count the number of tracks in each album*/
WITH tracks_per_album AS
    (SELECT DISTINCT t.album_id,
     COUNT(t.track_id) AS track_count
     FROM track AS t
     GROUP BY 1),

    /*#counting the number of tracks in the invoice orders*/     
    count_of_tracks AS
    (SELECT il.invoice_id,
            t.album_id,
            COUNT(il.track_id) AS track_count
     FROM track AS t
     INNER JOIN invoice_line AS il
     ON il.track_id = t.track_id
     GROUP BY 1,2)

SELECT purchase_type,
        COUNT(DISTINCT il.invoice_id) AS invoice_count,
        ROUND(CAST(COUNT(DISTINCT pt.invoice_id) AS FLOAT)/
             (SELECT COUNT (*)
             FROM invoice) *100,2) || '%' Percentage
FROM /*# A table of where invoice ids are albums purchases or not*/
    (SELECT il.invoice_id, 
            MIN(il.track_id),
    CASE WHEN
        ct.track_count = tpa.track_count THEN 'Album'
        ELSE 'Individual Tracks'
        END AS purchase_type
    FROM invoice_line AS il
    INNER JOIN count_of_tracks AS ct
    ON il.invoice_id = ct.invoice_id
    INNER JOIN tracks_per_album AS tpa
    ON ct.album_id = tpa.album_id
    GROUP BY il.invoice_id
       ) AS pt
INNER JOIN invoice_line AS il
ON pt.invoice_id = il.invoice_id
GROUP BY purchase_type  

Done.


purchase_type,invoice_count,Percentage
Album,117,19.06%
Individual Tracks,497,80.94%


The two methods gave different results.
Clearly there was an error.
Investigating this by joining the two tables and finding what is not matching.

In [22]:
%%sql
/*#count the number of tracks in each album*/
WITH tracks_per_album AS
    (SELECT DISTINCT t.album_id,
     COUNT(t.track_id) AS track_count
     FROM track AS t
     GROUP BY 1),

    /*#counting the number of tracks in the invoice orders*/     
    count_of_tracks AS
    (SELECT il.invoice_id,
            t.album_id,
            COUNT(il.track_id) AS track_count
     FROM track AS t
     INNER JOIN invoice_line AS il
     ON il.track_id = t.track_id
     GROUP BY 1,2),

table_a AS
(SELECT *,
         purchase_type,
        COUNT(DISTINCT il.invoice_id) AS invoice_count,
        ROUND(CAST(COUNT(DISTINCT pt.invoice_id) AS FLOAT)/
             (SELECT COUNT (*)
             FROM invoice) *100,2) || '%' Percentage
FROM /*# A table of where invoice ids are albums purchases or not*/
    (SELECT il.invoice_id, 
            MIN(il.track_id),
    CASE WHEN
        ct.track_count = tpa.track_count THEN 'Album'
        ELSE 'Individual Tracks'
        END AS purchase_type
    FROM invoice_line AS il
    INNER JOIN count_of_tracks AS ct
    ON il.invoice_id = ct.invoice_id
    INNER JOIN tracks_per_album AS tpa
    ON ct.album_id = tpa.album_id
    GROUP BY il.invoice_id
       ) AS pt
INNER JOIN invoice_line AS il
ON pt.invoice_id = il.invoice_id
GROUP BY il.invoice_id),

track_invoice AS
(SELECT t.album_id, il.track_id, il.invoice_id
FROM track AS t
INNER JOIN invoice_line AS il ON il.track_id = t.track_id),

table_b AS
(SELECT *,
         purchase_type2,
        COUNT(DISTINCT invoice_id) AS Number_Of_Invoices,
        ROUND((CAST (COUNT(DISTINCT invoice_id) AS FLOAT)*100)/ 
            (SELECT count(distinct invoice_id) FROM invoice),2) AS Percentage

FROM
(SELECT invoice_id,
        CASE
        WHEN (SELECT t2.track_id /*#These are tracks for the whole album*/
             FROM track AS t2
             WHERE t2.album_id = track_invoice2.album_id
             EXCEPT
             SELECT il2.track_id  /*#These are only the purchased tracks*/
             FROM invoice_line AS il2
             WHERE il2.invoice_id = track_invoice2.invoice_id
             ) IS NULL
            AND 
/*# Combining the reverse of these CASE statements will label each invoice*/
/*#where the purchased tracks in the invoice was the whole album*/
            (SELECT il2.track_id  
             FROM invoice_line AS il2
             WHERE il2.invoice_id = track_invoice2.invoice_id
             EXCEPT SELECT t2.track_id 
             FROM track AS t2
             WHERE t2.album_id = track_invoice2.album_id
            ) IS NULL
            THEN 'Album'
            ELSE 'Individual Tracks'
            END AS purchase_type2
    FROM track_invoice As track_invoice2
    )
GROUP BY invoice_id) 

SELECT table_a.invoice_id,
        table_b.invoice_id,
        table_a.purchase_type,
        table_b.purchase_type2

FROM table_a
LEFT JOIN table_b ON table_a.invoice_id = table_b.invoice_id
WHERE table_a.purchase_type != table_b.purchase_type2
ORDER BY purchase_type


Done.


invoice_id,invoice_id_1,purchase_type,purchase_type2
106,106,Album,Individual Tracks
127,127,Album,Individual Tracks
288,288,Album,Individual Tracks


The results of these 3 invoices dont match. 

But why?

In [23]:
%%sql
WITH All_invoice_tracks AS 
(SELECT il.invoice_id,
        t.track_id,
        t.album_id,
        t.name
FROM invoice_line AS il
INNER JOIN track AS t ON t.track_id = il.track_id
WHERE invoice_id IN ('106','127','288'))

SELECT t.track_id,
        t.album_id,
        t.name,
        ait.invoice_id
FROM track AS t
LEFT JOIN All_invoice_tracks AS ait ON t.track_id = ait.track_id
/* #There were too many rows to be of use so a bit of hardcoding below reduces this*/
WHERE t.album_id IN ('2','7','39','88','120','141','149','150','151','163','180','183','190','203','207','218','243','321','322')
ORDER BY 2,1

Done.


track_id,album_id,name,invoice_id
2,2,Balls to the Wall,106
2,2,Balls to the Wall,127
2,2,Balls to the Wall,288
51,7,We Die Young,None
52,7,Man In The Box,None
53,7,Sea Of Sorrow,288
54,7,Bleed The Freak,None
55,7,I Can't Remember,None
56,7,"Love, Hate, Love",None
57,7,It Ain't Like That,None


Here we have the answer.
These 3 invoices contain both album purchases (of album_id = 2),
and individual track purchases.
The except function treats these as individual track purchases.
Also given that the album is just one track its safe to call these individual track purchases.



### Who are the 10 most popular artist in the playlists

This will require joining many databases to get the information
First lets have a look at the types of playlists

In [24]:
%%sql
SELECT *
FROM playlist

Done.


playlist_id,name
1,Music
2,Movies
3,TV Shows
4,Audiobooks
5,90’s Music
6,Audiobooks
7,Movies
8,Music
9,Music Videos
10,TV Shows


In [25]:
%%sql
SELECT ar.name,
        COUNT(ar.name) AS total_times_in_playlist,
        ge.name
FROM playlist_track AS pt
LEFT JOIN track AS t ON t.track_id = pt.track_id
LEFT JOIN album AS al ON al.album_id = t.album_id
LEFT JOIN artist AS ar ON ar.artist_id = al.artist_id
LEFT JOIN genre AS ge ON ge.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10


Done.


name,total_times_in_playlist,name_1
Iron Maiden,516,Rock
U2,333,Pop
Metallica,296,Metal
Led Zeppelin,252,Rock
Deep Purple,226,Rock
Lost,184,Drama
Pearl Jam,177,Rock
Eric Clapton,145,Latin
Faith No More,145,Alternative & Punk
Lenny Kravitz,143,Metal


Top artists tend to be Rock and Metal.

### What proportion of tracks go unpurchased
Unbought tracks is potential buisness loss for Chinook

In [26]:
%%sql
WITH tracks_bought AS
        (SELECT t.track_id
         FROM invoice_line AS il
         LEFT JOIN track AS t 
         ON il.track_id = t.track_id),
     all_tracks_count AS
        (SELECT  t.track_id
         FROM track AS t
        LEFT JOIN invoice_line AS il 
         ON il.track_id = t.track_id)

SELECT  COUNT(*) AS purchased_tracks,
        (SELECT COUNT (*) FROM all_tracks_count) AS total_tracks
FROM tracks_bought

Done.


purchased_tracks,total_tracks
4757,6454


73% of the tracks in the market were purchased. Clearly the company is doing well in deciding which tracks to purchase.
But what are the worst business decisions?
Let look at which artists are not selling.

In [27]:
%%sql
WITH unbought_tracks AS
     (SELECT t.track_id FROM track AS t
     LEFT JOIN invoice_line AS il ON il.track_id = t.track_id
    EXCEPT 
    SELECT il.track_id FROM invoice_line AS il
     LEFT JOIN track AS t ON t.track_id = il.track_id)
    
SELECT ar.name AS artist,
        al.title AS album,
        COUNT(t.name) AS number_of_tracks,
        ge.name AS genre
FROM unbought_tracks AS ut
LEFT JOIN track AS t ON t.track_id = ut.track_id
LEFT JOIN album AS al ON al.album_id = t.album_id
LEFT JOIN artist AS ar ON ar.artist_id = al.artist_id
LEFT JOIN genre AS ge ON ge.genre_id = t.genre_id
GROUP BY 1,2
ORDER BY 3 DESC
LIMIT 20;

Done.


artist,album,number_of_tracks,genre
Lenny Kravitz,Greatest Hits,33,Metal
Lost,"Lost, Season 3",26,Drama
Chico Buarque,Minha Historia,25,Latin
The Office,"The Office, Season 3",25,Comedy
Battlestar Galactica (Classic),"Battlestar Galactica (Classic), Season 1",24,Sci Fi & Fantasy
Lost,"Lost, Season 1",24,TV Shows
Lost,"Lost, Season 2",24,TV Shows
Heroes,"Heroes, Season 1",23,Drama
Gene Krupa,Up An' Atom,22,Jazz
The Office,"The Office, Season 2",22,TV Shows


Latin music and tv-show sound tracks make up the greatest amount of unsold music.

### Do protected vs non-protected media types have an effect on purchases

Clearly there is more freedom of use for the customer in a non-protected track but is this actually influencing purchasing decisions?

First lets see what types of tracks there are.

In [28]:
%%sql
SELECT * FROM media_type

Done.


media_type_id,name
1,MPEG audio file
2,Protected AAC audio file
3,Protected MPEG-4 video file
4,Purchased AAC audio file
5,AAC audio file


There are only two media_types we are interested in:

Protected AAC audio file and Protected MPEG-4 video file

In [29]:
%%sql
WITH total_tracks AS
(SELECT COUNT(*) AS available_tracks,
         t2.media_type_id AS media_type_id
FROM Track AS t2
GROUP by 2
)

SELECT mt.name AS name,
        COUNT(t.track_id) AS number_of_purchases,
        tt.available_tracks
FROM invoice_line AS il
LEFT JOIN track AS t ON t.track_id = il.track_id
LEFT JOIN media_type AS mt on mt.media_type_id = t.media_type_id
LEFT JOIN total_tracks AS tt ON tt.media_type_id = mt.media_type_id
GROUP BY mt.name

Done.


name,number_of_purchases,available_tracks
AAC audio file,21,11
MPEG audio file,4259,3034
Protected AAC audio file,439,237
Protected MPEG-4 video file,3,214
Purchased AAC audio file,35,7


There is mixed information here. 

MPEG is the most popular and the protected MPEG purchase are insignificant.

For AAC audio types the majority is in the protected category. The unprotected AAC files in the Chinook market  is only 11 tracks but is being sold at almost twice the rate. A near similar trend is seen in the purchased and protected AAC audio files. Perhaps if Chinook invested more in AAC they could see a higher rate of return


### Conclusion

Chinook should prioritise purchasing the rights of popular singles over albums. ALbums should not be disregarded however as ~20% of sales is a significant market

Chinooks employees are performing at similar rates and have near equal access to customers from the top markets.

Chinooks top 10 market genres are:Rock, Alternative & Punk,Metal, R&B/Soul Blues, Alternative Hip Hop/Rap, Pop, Latin and Jazz. (Rock at No.1 to Jazz at No.10)

Chinooks single sale markets is significant with a comparable market value to the biggest market - The USA

Chinooks top selling artists tend to be be Rock and Metal.

Chinooks Latin music and tv-show sound tracks make up the greatest amount of unsold music.

Chinooks protected MPEG audio files are likely a waste of money with low number of purchases to tracks owned. Unprotected MPEG audio files makes up the largest share of purchase but the return on investment is likely to be better with AAC files.